# 🎵 Music Genre Clustering - Feature Extraction & Analysis

## Deep Mathematical Implementation and Exploratory Data Analysis

**Project:** Music Genre Clustering using K-Means & GMM  
**Dataset:** GTZAN (1,000 songs, 10 genres)  
**Author:** Vedant  
**Date:** October 2025

---

## Notebook Overview

This notebook covers:
1. Audio feature extraction (5 features)
2. Mathematical foundations of each feature
3. Exploratory Data Analysis (EDA)
4. Feature correlation analysis
5. Data preprocessing and standardization

## 1. Import Libraries and Setup

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import librosa
import librosa.display

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec

# Utilities
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context('notebook', font_scale=1.2)
sns.set_palette('husl')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("\n" + "="*70)
print("LIBRARY VERSIONS")
print("="*70)
print(f"NumPy:     {np.__version__}")
print(f"Pandas:    {pd.__version__}")
print(f"Librosa:   {librosa.__version__}")
print(f"Matplotlib: {plt.matplotlib.__version__}")
print(f"Seaborn:   {sns.__version__}")
print("="*70)
print("✅ All libraries loaded successfully!\n")

## 2. Mathematical Background of Audio Features

### 2.1 Feature Definitions

#### **1. Tempo (BPM)**

Tempo measures the speed of music in beats per minute:

$$\text{Tempo (BPM)} = \frac{60 \times \text{Sample Rate}}{\text{Hop Length} \times \text{Beat Period}}$$

Extracted using librosa's beat tracking algorithm which:
1. Computes onset strength envelope
2. Applies autocorrelation to find periodicity
3. Estimates tempo from dominant period

---

#### **2. Energy (RMS)**

Root Mean Square energy measures signal intensity:

$$E_{RMS}(t) = \sqrt{\frac{1}{N} \sum_{n=0}^{N-1} |x[n]|^2}$$

Where:
- $x[n]$ = audio signal samples
- $N$ = frame length
- Higher RMS = louder, more energetic sound

---

#### **3. Loudness (RMS)**

For GTZAN dataset, loudness is equivalent to energy (RMS mean):

$$\text{Loudness} = \frac{1}{T} \sum_{t=1}^{T} E_{RMS}(t)$$

Where $T$ = total number of frames

---

#### **4. Valence (Approximated)**

Valence represents musical positiveness/happiness. Approximated using:

$$\text{Valence} \approx \frac{SC_{norm} + H_{norm}}{2}$$

Where:
- $SC_{norm} = \frac{\text{Spectral Centroid}}{4000}$ (normalized)
- $H_{norm} = \frac{\text{Harmonic Content}}{0.5}$ (normalized)

**Spectral Centroid:**
$$SC = \frac{\sum_{k=1}^{K} f(k) \cdot |X(k)|}{\sum_{k=1}^{K} |X(k)|}$$

Represents the "center of mass" of the spectrum (brightness)

---

#### **5. Danceability (Approximated)**

Danceability measures how suitable a track is for dancing:

$$\text{Danceability} \approx \frac{T_{norm} + ZCR_{mean}}{2}$$

Where:
- $T_{norm} = \frac{\text{Tempo}}{200}$ (normalized by max expected BPM)
- $ZCR_{mean}$ = Zero Crossing Rate (mean)

**Zero Crossing Rate:**
$$ZCR = \frac{1}{N-1} \sum_{n=1}^{N-1} \mathbb{1}_{x[n] \cdot x[n-1] < 0}$$

Measures how often the signal crosses zero amplitude

## 3. Feature Extraction Class Implementation

In [ ]:
class AudioFeatureExtractor:
    """
    Comprehensive audio feature extraction for music clustering.
    
    Features:
    ---------
    1. Tempo (BPM) - Beat tracking
    2. Energy (RMS) - Root Mean Square
    3. Loudness (RMS) - Mean energy
    4. Valence - Spectral centroid + harmonic content
    5. Danceability - Tempo + zero crossing rate
    
    Parameters:
    -----------
    audio_dir : str or Path
        Directory containing GTZAN audio files
    duration : int, default=30
        Duration to analyze (seconds)
    sr : int, default=22050
        Sample rate (Hz)
    """
    
    def __init__(self, audio_dir, duration=30, sr=22050):
        self.audio_dir = Path(audio_dir)
        self.duration = duration
        self.sr = sr
        print(f"\n📁 Audio directory: {self.audio_dir}")
        print(f"⏱️  Duration: {self.duration}s")
        print(f"🎼 Sample rate: {self.sr} Hz\n")
        
    def extract_tempo(self, y, sr):
        """
        Extract tempo using beat tracking.
        
        Algorithm:
        ----------
        1. Compute onset strength envelope
        2. Apply autocorrelation
        3. Find dominant periodic component
        4. Convert to BPM
        """
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        return float(tempo)
    
    def extract_energy_loudness(self, y):
        """
        Extract RMS energy/loudness.
        
        Formula:
        --------
        RMS = sqrt(mean(x^2))
        """
        rms = librosa.feature.rms(y=y)
        return float(np.mean(rms))
    
    def extract_valence(self, y, sr):
        """
        Approximate valence from spectral and harmonic features.
        
        Components:
        -----------
        1. Spectral Centroid (brightness) - higher = brighter/happier
        2. Harmonic Content - stronger harmonics = more musical/positive
        
        Formula:
        --------
        valence = (centroid_norm + harmonic_norm) / 2
        """
        # Spectral centroid
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        centroid_mean = np.mean(spectral_centroid)
        centroid_norm = centroid_mean / 4000.0  # Normalize (typical max ~4000 Hz)
        
        # Harmonic component
        harmonic, _ = librosa.effects.hpss(y)
        harmonic_mean = np.mean(np.abs(harmonic))
        harmonic_norm = harmonic_mean / 0.5  # Normalize (typical max ~0.5)
        
        # Combine
        valence = (centroid_norm + harmonic_norm) / 2.0
        return float(valence)
    
    def extract_danceability(self, y, sr, tempo):
        """
        Approximate danceability from tempo and rhythm.
        
        Components:
        -----------
        1. Tempo - faster = more danceable
        2. Zero Crossing Rate - rhythmic complexity
        
        Formula:
        --------
        danceability = (tempo_norm + zcr_mean) / 2
        """
        # Tempo component
        tempo_norm = tempo / 200.0  # Normalize (typical max ~200 BPM)
        
        # Zero crossing rate
        zcr = librosa.feature.zero_crossing_rate(y)
        zcr_mean = float(np.mean(zcr))
        
        # Combine
        danceability = (tempo_norm + zcr_mean) / 2.0
        return float(danceability)
    
    def extract_single_file(self, file_path):
        """
        Extract all 5 features from a single audio file.
        
        Returns:
        --------
        dict : Dictionary containing all features
        """
        try:
            # Load audio
            y, sr = librosa.load(file_path, duration=self.duration, sr=self.sr)
            
            # Extract features
            tempo = self.extract_tempo(y, sr)
            energy = self.extract_energy_loudness(y)
            loudness = energy  # Same as energy for GTZAN
            valence = self.extract_valence(y, sr)
            danceability = self.extract_danceability(y, sr, tempo)
            
            return {
                'filename': file_path.name,
                'genre': file_path.parent.name,
                'tempo': tempo,
                'energy': energy,
                'loudness': loudness,
                'valence': valence,
                'danceability': danceability
            }
            
        except Exception as e:
            print(f"❌ Error processing {file_path.name}: {e}")
            return None
    
    def extract_all(self, save_path=None):
        """
        Extract features from entire dataset.
        
        Parameters:
        -----------
        save_path : str or Path, optional
            Path to save extracted features CSV
        
        Returns:
        --------
        DataFrame : Features for all songs
        """
        features_list = []
        audio_files = sorted(list(self.audio_dir.glob('**/*.wav')))
        
        print(f"\n🎵 Found {len(audio_files)} audio files")
        print("\n" + "="*70)
        print("EXTRACTING FEATURES")
        print("="*70 + "\n")
        
        for audio_file in tqdm(audio_files, desc="Processing", unit="file"):
            features = self.extract_single_file(audio_file)
            if features:
                features_list.append(features)
        
        # Create DataFrame
        df = pd.DataFrame(features_list)
        
        # Handle missing values (zero-imputation)
        df.fillna(0, inplace=True)
        
        print(f"\n✅ Extraction complete!")
        print(f"   Total songs: {len(df)}")
        print(f"   Total features: {len(df.columns) - 2}\n")  # Exclude filename and genre
        
        # Save if path provided
        if save_path:
            save_path = Path(save_path)
            save_path.parent.mkdir(parents=True, exist_ok=True)
            df.to_csv(save_path, index=False)
            print(f"💾 Saved to: {save_path}\n")
        
        return df

print("✅ AudioFeatureExtractor class defined successfully!")

## 4. Extract Features from GTZAN Dataset

**Note:** Uncomment and run the cell below if you have GTZAN dataset locally.  
Otherwise, load pre-extracted features.

In [ ]:
# # OPTION 1: Extract features from raw audio (if you have GTZAN dataset)
# extractor = AudioFeatureExtractor(
#     audio_dir='data/gtzan/genres_original',
#     duration=30,
#     sr=22050
# )

# features_df = extractor.extract_all(save_path='data/processed/features_selected.csv')

# OPTION 2: Load pre-extracted features
features_df = pd.read_csv('../data/processed/features_selected.csv')

print("\n" + "="*70)
print("DATASET LOADED")
print("="*70)
print(f"Shape: {features_df.shape}")
print(f"Columns: {features_df.columns.tolist()}")
print("="*70 + "\n")

# Display first 10 rows
features_df.head(10)

## 5. Exploratory Data Analysis (EDA)

### 5.1 Statistical Summary

In [ ]:
# Define feature columns
feature_cols = ['tempo', 'energy', 'loudness', 'valence', 'danceability']

print("\n" + "="*70)
print("STATISTICAL SUMMARY")
print("="*70 + "\n")

# Compute statistics
stats = features_df[feature_cols].describe()
print(stats)

# Additional statistics
print("\n" + "="*70)
print("ADDITIONAL STATISTICS")
print("="*70)

for col in feature_cols:
    print(f"\n{col.upper()}:")
    print(f"  Range: [{features_df[col].min():.4f}, {features_df[col].max():.4f}]")
    print(f"  IQR: {features_df[col].quantile(0.75) - features_df[col].quantile(0.25):.4f}")
    print(f"  Variance: {features_df[col].var():.4f}")
    print(f"  Skewness: {features_df[col].skew():.4f}")
    print(f"  Kurtosis: {features_df[col].kurtosis():.4f}")

### 5.2 Missing Values and Data Quality

In [ ]:
print("\n" + "="*70)
print("DATA QUALITY CHECK")
print("="*70)

# Missing values
print("\nMissing Values:")
missing = features_df.isnull().sum()
print(missing)
print(f"\nTotal missing values: {missing.sum()}")

# Duplicates
duplicates = features_df.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

# Genre distribution
print("\n" + "="*70)
print("GENRE DISTRIBUTION")
print("="*70 + "\n")
genre_counts = features_df['genre'].value_counts().sort_index()
print(genre_counts)

# Check balance
print(f"\nDataset balance:")
print(f"  Min: {genre_counts.min()} songs")
print(f"  Max: {genre_counts.max()} songs")
print(f"  Balanced: {'✅ Yes' if genre_counts.std() < 5 else '❌ No'}")

### 5.3 Feature Distributions Visualization

In [ ]:
# Create comprehensive distribution plots
fig, axes = plt.subplots(3, 2, figsize=(16, 12))
fig.suptitle('Feature Distributions with Statistical Annotations', 
             fontsize=16, fontweight='bold', y=0.995)

for idx, feature in enumerate(feature_cols):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]
    
    # Histogram
    n, bins, patches = ax.hist(features_df[feature], bins=40, 
                                edgecolor='black', alpha=0.7, color='skyblue')
    
    # Statistical lines
    mean_val = features_df[feature].mean()
    median_val = features_df[feature].median()
    std_val = features_df[feature].std()
    
    ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, 
               label=f'Mean: {mean_val:.3f}')
    ax.axvline(median_val, color='green', linestyle='--', linewidth=2, 
               label=f'Median: {median_val:.3f}')
    ax.axvline(mean_val + std_val, color='orange', linestyle=':', linewidth=1.5,
               label=f'±1σ: {std_val:.3f}')
    ax.axvline(mean_val - std_val, color='orange', linestyle=':', linewidth=1.5)
    
    # Formatting
    ax.set_title(f'{feature.capitalize()} Distribution', fontweight='bold', fontsize=12)
    ax.set_xlabel(feature.capitalize(), fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3, linestyle=':')

# Remove last empty subplot
fig.delaxes(axes[2, 1])

plt.tight_layout()
plt.show()

print("\n📊 Distribution plots generated successfully!")

### 5.4 Box Plots for Outlier Detection

In [ ]:
# Box plots
fig, axes = plt.subplots(1, 5, figsize=(20, 5))
fig.suptitle('Box Plots - Outlier Detection', fontsize=16, fontweight='bold')

for idx, feature in enumerate(feature_cols):
    ax = axes[idx]
    
    # Create box plot
    box_data = ax.boxplot([features_df[feature]], 
                           labels=[feature.capitalize()],
                           patch_artist=True,
                           notch=True,
                           showmeans=True,
                           meanprops=dict(marker='D', markerfacecolor='red', markersize=8))
    
    # Color boxes
    for patch in box_data['boxes']:
        patch.set_facecolor('lightblue')
    
    # Calculate outliers
    Q1 = features_df[feature].quantile(0.25)
    Q3 = features_df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = features_df[(features_df[feature] < lower_bound) | 
                          (features_df[feature] > upper_bound)][feature]
    
    # Annotations
    ax.set_ylabel('Value', fontsize=10)
    ax.set_title(f'{len(outliers)} outliers', fontsize=10, color='red')
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n📊 Box plots generated successfully!")

### 5.5 Correlation Analysis

In [ ]:
# Compute correlation matrix
correlation_matrix = features_df[feature_cols].corr()

print("\n" + "="*70)
print("CORRELATION MATRIX")
print("="*70 + "\n")
print(correlation_matrix)

# Visualize correlation heatmap
fig, ax = plt.subplots(figsize=(10, 8))

# Create heatmap
sns.heatmap(correlation_matrix, 
            annot=True, 
            fmt='.3f',
            cmap='coolwarm', 
            center=0,
            square=True, 
            linewidths=1,
            cbar_kws={"shrink": 0.8, "label": "Correlation Coefficient"},
            vmin=-1, 
            vmax=1,
            ax=ax)

ax.set_title('Feature Correlation Heatmap', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

# Interpret correlations
print("\n" + "="*70)
print("CORRELATION INTERPRETATION")
print("="*70)

# Find strong correlations (|r| > 0.7)
strong_corr = []
for i in range(len(feature_cols)):
    for j in range(i+1, len(feature_cols)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.7:
            strong_corr.append((feature_cols[i], feature_cols[j], corr_val))

if strong_corr:
    print("\nStrong Correlations (|r| > 0.7):")
    for feat1, feat2, corr in strong_corr:
        print(f"  {feat1} ↔ {feat2}: {corr:.4f}")
else:
    print("\n✅ No strong correlations found - features are independent!")

print("\nKey Observations:")
print(f"  • Energy-Loudness correlation: {correlation_matrix.loc['energy', 'loudness']:.4f} (Expected: ~1.0)")
print(f"  • Tempo-Danceability correlation: {correlation_matrix.loc['tempo', 'danceability']:.4f}")
print(f"  • Low correlations indicate good feature diversity for clustering")

### 5.6 Pair Plot - Relationships Between Features

In [ ]:
# Create pair plot (sample subset for performance)
sample_size = min(1000, len(features_df))
sample_df = features_df.sample(n=sample_size, random_state=42)

print(f"\n📊 Generating pair plot for {sample_size} samples...\n")

# Pair plot
pairplot = sns.pairplot(sample_df, 
                         vars=feature_cols,
                         hue='genre',
                         diag_kind='kde',
                         plot_kws={'alpha': 0.6, 's': 30},
                         height=2.5)

pairplot.fig.suptitle('Feature Relationships by Genre', 
                       y=1.01, fontsize=16, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✅ Pair plot generated successfully!")

### 5.7 Genre-wise Feature Analysis

In [ ]:
# Compute mean features per genre
genre_means = features_df.groupby('genre')[feature_cols].mean()

print("\n" + "="*70)
print("MEAN FEATURES BY GENRE")
print("="*70 + "\n")
print(genre_means)

# Visualize with heatmap
fig, ax = plt.subplots(figsize=(12, 8))

sns.heatmap(genre_means.T, 
            annot=True, 
            fmt='.2f',
            cmap='YlOrRd',
            linewidths=1,
            cbar_kws={"label": "Feature Value"},
            ax=ax)

ax.set_title('Average Feature Values by Genre', fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Genre', fontsize=12)
ax.set_ylabel('Feature', fontsize=12)

plt.tight_layout()
plt.show()

print("\n✅ Genre analysis heatmap generated!")

## 6. Data Preprocessing for Clustering

### 6.1 Standardization (Z-score Normalization)

**Mathematical Formula:**

$$z = \frac{x - \mu}{\sigma}$$

Where:
- $x$ = original feature value
- $\mu$ = mean of feature
- $\sigma$ = standard deviation of feature
- $z$ = standardized value (mean=0, std=1)

**Why Standardization?**
1. Features have different scales (Tempo: 50-200, Energy: 0-0.5)
2. K-Means uses Euclidean distance - sensitive to scale
3. Standardization ensures equal contribution from all features

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize scaler
scaler = StandardScaler()

# Standardize features
X = features_df[feature_cols].values
X_scaled = scaler.fit_transform(X)

print("\n" + "="*70)
print("STANDARDIZATION RESULTS")
print("="*70)

# Create DataFrame for scaled features
scaled_df = pd.DataFrame(X_scaled, columns=[f'{col}_scaled' for col in feature_cols])

# Verify standardization
print("\nOriginal Statistics:")
print(features_df[feature_cols].describe().loc[['mean', 'std']])

print("\nStandardized Statistics:")
print(scaled_df.describe().loc[['mean', 'std']])

print("\n✅ Standardization complete!")
print("   All features now have mean ≈ 0 and std ≈ 1")

### 6.2 Save Processed Data

In [ ]:
# Save processed features
output_path = Path('data/processed/features_standardized.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)

# Combine original metadata with scaled features
processed_df = pd.concat([
    features_df[['filename', 'genre']],
    features_df[feature_cols],
    scaled_df
], axis=1)

processed_df.to_csv(output_path, index=False)

print(f"\n💾 Processed data saved to: {output_path}")
print(f"   Shape: {processed_df.shape}")
print(f"   Columns: {processed_df.columns.tolist()}")

## 7. Summary and Key Findings

### Dataset Overview
- **Total Songs:** 1,000
- **Genres:** 10 (balanced)
- **Features:** 5 audio characteristics
- **Missing Values:** None

### Feature Characteristics
1. **Tempo:** Wide range (50-200 BPM), relatively uniform distribution
2. **Energy/Loudness:** Low to moderate values, right-skewed
3. **Valence:** Moderate values, approximately normal distribution
4. **Danceability:** Moderate to high values, slightly left-skewed

### Correlation Insights
- Energy and Loudness highly correlated (r ≈ 1.0) - expected
- Other features show low correlation - good for clustering
- Features provide complementary information

### Next Steps
1. Apply K-Means clustering (Notebook 2)
2. Apply GMM clustering (Notebook 3)
3. Perform PCA dimensionality reduction (Notebook 4)
4. Evaluate clustering performance (Notebook 5)